In [ ]:
import torch
from torch import nn
import numpy as np
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
df = pd.read_csv("Student_Performance.csv") # Reading and saving dataframe
features = ["Hours Studied", "Previous Scores", "Extracurricular Activities", "Sleep Hours", "Sample Question Papers Practiced"]
target = "Performance Index"

df["Extracurricular Activities"] = df["Extracurricular Activities"].map({'Yes': 1, 'No': 0})
X = df[features].values
y = df[target].values.reshape(len(df[target].values), 1) # Equivalent y = df[target].values.reshape(-1, 1)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
print("Features: ", X)
print("Features size: ", X.shape)
print()
print("Target: ", y)
print("Target size: ", y.shape)

Features:  tensor([[ 7., 99.,  1.,  9.,  1.],
        [ 4., 82.,  0.,  4.,  2.],
        [ 8., 51.,  1.,  7.,  2.],
        ...,
        [ 6., 83.,  1.,  8.,  5.],
        [ 9., 97.,  1.,  7.,  0.],
        [ 7., 74.,  0.,  8.,  1.]])
Features size:  torch.Size([10000, 5])

Target:  tensor([[91.],
        [65.],
        [45.],
        ...,
        [74.],
        [95.],
        [64.]])
Target size:  torch.Size([10000, 1])


In [ ]:
train_split = int(len(X) * 0.8)
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

print((len(X_train), len(y_train), len(X_test), len(y_test)))

(8000, 8000, 2000, 2000)


In [ ]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.w1 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w2 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w3 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w4 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.w5 = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
    self.bias = nn.Parameter(torch.randn(1,
                                    requires_grad=True,
                                    dtype=torch.float))
  def forward(self, hrs, prevScores, extraCurry, sleepHrs, sampleQp):
    return hrs * self.w1 + prevScores * self.w2 + extraCurry * self.w3 + sleepHrs * self.w4 + sampleQp * self.w5 + self.bias

torch.manual_seed(42)
model_0 = LinearRegression()
model_0.state_dict()

OrderedDict([('w1', tensor([0.3367])),
             ('w2', tensor([0.1288])),
             ('w3', tensor([0.2345])),
             ('w4', tensor([0.2303])),
             ('w5', tensor([-1.1229])),
             ('bias', tensor([-0.1863]))])

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                            lr=0.001)

# Standardize 'Previous Scores' using the mean and standard deviation from the training data
mean_prev_scores = X_train[:, 1].mean()
std_prev_scores = X_train[:, 1].std()

X_train[:, 1] = (X_train[:, 1] - mean_prev_scores) / std_prev_scores
X_test[:, 1] = (X_test[:, 1] - mean_prev_scores) / std_prev_scores

print("Standardized X_train (first 5 rows of 'Previous Scores'):", X_train[:5, 1])
print("Standardized X_test (first 5 rows of 'Previous Scores'):", X_test[:5, 1])

Standardized X_train (first 5 rows of 'Previous Scores'): tensor([ 1.7030,  0.7236, -1.0621, -1.0045,  0.3204])
Standardized X_test (first 5 rows of 'Previous Scores'): tensor([-1.1197, -1.2350,  0.4356,  0.9541,  0.7813])


In [ ]:
epochs = 500
for epoch in range(epochs):
  model_0.train()
  # 1. Forward pass
  train_preds = model_0.forward(X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3], X_train[:, 4])

  # 2. Compute loss
  train_loss = loss_fn(train_preds, y_train)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. loss backward
  train_loss.backward()

  # 5. Optimizer step
  optimizer.step()

  model_0.eval()

  with torch.inference_mode():
    test_preds = model_0.forward(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4])
    test_loss = loss_fn(test_preds, y_test)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch} | Train loss: {train_loss} | Test loss: {test_loss}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([8000, 1])) that is different to the input size (torch.Size([8000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([2000, 1])) that is different to the input size (torch.Size([2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 0 | Train loss: 57.19602584838867 | Test loss: 57.47905731201172
Epoch: 10 | Train loss: 57.02028274536133 | Test loss: 57.30193328857422
Epoch: 20 | Train loss: 56.84455490112305 | Test loss: 57.12480926513672
Epoch: 30 | Train loss: 56.66881561279297 | Test loss: 56.94768142700195
Epoch: 40 | Train loss: 56.49308776855469 | Test loss: 56.77056121826172
Epoch: 50 | Train loss: 56.31734848022461 | Test loss: 56.593441009521484
Epoch: 60 | Train loss: 56.1416130065918 | Test loss: 56.416316986083984
Epoch: 70 | Train loss: 55.96588134765625 | Test loss: 56.23918914794922
Epoch: 80 | Train loss: 55.79014587402344 | Test loss: 56.06206130981445
Epoch: 90 | Train loss: 55.61441421508789 | Test loss: 55.88494110107422


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([2000, 1])) that is different to the input size (torch.Size([2000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([8000, 1])) that is different to the input size (torch.Size([8000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 100 | Train loss: 55.43867874145508 | Test loss: 55.70781326293945
Epoch: 110 | Train loss: 55.262939453125 | Test loss: 55.53069305419922
Epoch: 120 | Train loss: 55.08720779418945 | Test loss: 55.35356521606445
Epoch: 130 | Train loss: 54.911476135253906 | Test loss: 55.17643737792969
Epoch: 140 | Train loss: 54.735748291015625 | Test loss: 54.99932098388672
Epoch: 150 | Train loss: 54.56001281738281 | Test loss: 54.82219696044922
Epoch: 160 | Train loss: 54.38428497314453 | Test loss: 54.64508056640625
Epoch: 170 | Train loss: 54.208553314208984 | Test loss: 54.467952728271484
Epoch: 180 | Train loss: 54.03282165527344 | Test loss: 54.29083251953125
Epoch: 190 | Train loss: 53.85708236694336 | Test loss: 54.113712310791016
Epoch: 200 | Train loss: 53.68135452270508 | Test loss: 53.93658447265625
Epoch: 210 | Train loss: 53.50562286376953 | Test loss: 53.75946807861328
Epoch: 220 | Train loss: 53.329891204833984 | Test loss: 53.58234405517578
Epoch: 230 | Train loss: 53.154159

In [ ]:
with torch.inference_mode():
  print(model_0.forward(torch.tensor(7), torch.tensor(99), torch.tensor(1), torch.tensor(9), torch.tensor(1)))